<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/SQLCourse2_Setup_with_LOAD_command.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

Tutorial based on [SQLCourse2](https://www.sqlcourse2.com/) <br>
Data is in this [Google Sheet](https://docs.google.com/spreadsheets/d/1m_LE_ahVHY0PrXmtyVVjO5CyqvmOMOwGT6hPYBYHBqw/edit?usp=sharing) <br>
The data needs to be downloaded as two CSV files from the Google Docs above

#Using Local Server

## Install MySQL

In [1]:
!apt update > /dev/null
!apt install -y  mysql-server > /dev/null
!/etc/init.d/mysql restart
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'





 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


## Create Tables

In [2]:
!mysql praxisDB -e 'drop table if exists itemsordered'
# --------------------------------------------------------------
!mysql praxisDB -e '\
CREATE TABLE IF NOT EXISTS itemsordered (\
customerid char(5),orderdate date, item varchar(30), quantity int(4), price decimal(5,2)) \
'

In [3]:
!mysql praxisDB -e 'desc itemsordered'

+------------+--------------+------+-----+---------+-------+
| Field      | Type         | Null | Key | Default | Extra |
+------------+--------------+------+-----+---------+-------+
| customerid | char(5)      | YES  |     | NULL    |       |
| orderdate  | date         | YES  |     | NULL    |       |
| item       | varchar(30)  | YES  |     | NULL    |       |
| quantity   | int(4)       | YES  |     | NULL    |       |
| price      | decimal(5,2) | YES  |     | NULL    |       |
+------------+--------------+------+-----+---------+-------+


In [4]:
!mysql praxisDB -e 'drop table if exists customers'
# --------------------------------------------------------------
!mysql praxisDB -e '\
CREATE TABLE IF NOT EXISTS customers (customerid char(5),firstname varchar(20),lastname varchar(20), city varchar(20), state varchar(20)) \
'

In [5]:
!mysql praxisDB -e 'desc customers'

+------------+-------------+------+-----+---------+-------+
| Field      | Type        | Null | Key | Default | Extra |
+------------+-------------+------+-----+---------+-------+
| customerid | char(5)     | YES  |     | NULL    |       |
| firstname  | varchar(20) | YES  |     | NULL    |       |
| lastname   | varchar(20) | YES  |     | NULL    |       |
| city       | varchar(20) | YES  |     | NULL    |       |
| state      | varchar(20) | YES  |     | NULL    |       |
+------------+-------------+------+-----+---------+-------+


## Load Tables

In [6]:
# Download two CSV files from G-Drive into the Colab VM
!gdown https://drive.google.com/uc?id=1WxuruueFwMTjmPb-DvvVnGYB78Ty4OkP
!gdown https://drive.google.com/uc?id=1j49sqYHv0UoskVJDzNObWxECFF7ySsld

Downloading...
From: https://drive.google.com/uc?id=1WxuruueFwMTjmPb-DvvVnGYB78Ty4OkP
To: /content/SQLCourse2 - ItemsOrdered.csv
100% 1.16k/1.16k [00:00<00:00, 1.41MB/s]
Downloading...
From: https://drive.google.com/uc?id=1j49sqYHv0UoskVJDzNObWxECFF7ySsld
To: /content/SQLCourse2 - Customers.csv
100% 695/695 [00:00<00:00, 469kB/s]


In [7]:
# upload the two CSV files into the Colab VM
!ls

 sample_data  'SQLCourse2 - Customers.csv'  'SQLCourse2 - ItemsOrdered.csv'


In [8]:
# Note first line, with header columns ignored
# Note date is converted from string to proper date format
#
!mysql praxisDB -e "   \
LOAD DATA LOCAL INFILE 'SQLCourse2 - ItemsOrdered.csv' \
ignore \
INTO  TABLE itemsordered  \
FIELDS TERMINATED BY ',' enclosed by '\"' LINES TERMINATED BY '\r\n' IGNORE 1 LINES  \
(customerid,@orderdate, item, quantity, price) \
set orderdate = STR_TO_DATE(@orderdate,'%d-%b-%Y');\
"

In [10]:
#!mysql praxisDB -e 'select * from itemsordered;'

In [11]:
# Note first line, with header columns ignored
#
!mysql praxisDB -e "   \
LOAD DATA LOCAL INFILE 'SQLCourse2 - Customers.csv' INTO TABLE customers \
FIELDS TERMINATED BY ',' enclosed by '\"' LINES TERMINATED BY '\r\n'  IGNORE 1 LINES;  \
"

In [ ]:
#!mysql praxisDB -e 'select * from customers;'

#Using Remote Server

## Set Public Credentials

In [ ]:
# enter credentials here
#hostName = 'remotemysql.com'   # place your own credentials here
#userName = 'yourUserName'      # place your own credentials here
#passWord = 'yourPassWord'      # place your own credentials here
#dbName =  'yourDBname'         # place your own credentials here

#print(hostName,userName,dbName)

## Set Private Credentials

In [ ]:
#connect to G-Drive
from google.colab import drive
drive.mount('/content/drive')
# copy credentials file
#!ls /content/drive/'My Drive'/Praxis/WebCredentials
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/sqlCredentials_020221.py .
!cp /content/drive/'My Drive'/Praxis/WebCredentials/cleverCloud.py credentials.py
from credentials import hostName,userName,passWord,dbName

#for the sake of privacy
#the following credentials need to be stored in a text file called sqlCredentials.py
#in the format given below
#in the Colab VM


print(hostName,userName,passWord,dbName)

Mounted at /content/drive
bq3ojarbqgmkc7prtopv-mysql.services.clever-cloud.com uxnhrnbgcu47thlz LEEP91xTUEKq36h2DyzT bq3ojarbqgmkc7prtopv


In [ ]:
#NOTE SECURITY WARNING
#Passing passwords in the command line is inherently risky and SHOULD NOT BE USED for any non-trivial work
#---------------------------------------------------------------------
!mysql -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e 'show tables'

mysql: [Warning] Using a password on the command line interface can be insecure.
+--------------------------------+
| Tables_in_bq3ojarbqgmkc7prtopv |
+--------------------------------+
| Dept                           |
| Emp                            |
| customers                      |
| itemsordered                   |
+--------------------------------+


## Create Tables

In [ ]:
#NOTE SECURITY WARNING
#Passing passwords in the command line is inherently risky and SHOULD NOT BE USED for any non-trivial work
#---------------------------------------------------------------------
!mysql -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e 'drop table if exists customers'
# --------------------------------------------------------------
!mysql -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e '\
CREATE TABLE IF NOT EXISTS customers (\
customerid char(5),firstname varchar(20),lastname varchar(20), city varchar(20), state varchar(20)) \
'

mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
#NOTE SECURITY WARNING
#Passing passwords in the command line is inherently risky and SHOULD NOT BE USED for any non-trivial work
#---------------------------------------------------------------------
!mysql -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e 'drop table if exists itemsordered'
# --------------------------------------------------------------
!mysql -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e '\
CREATE TABLE IF NOT EXISTS itemsordered (\
customerid char(5),orderdate date, item varchar(30), quantity int(4), price decimal(7,2)) \
'

mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
#NOTE SECURITY WARNING
#Passing passwords in the command line is inherently risky and SHOULD NOT BE USED for any non-trivial work
#---------------------------------------------------------------------
!mysql -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e 'show tables'

mysql: [Warning] Using a password on the command line interface can be insecure.
+--------------------------------+
| Tables_in_bq3ojarbqgmkc7prtopv |
+--------------------------------+
| Dept                           |
| Emp                            |
| customers                      |
| itemsordered                   |
+--------------------------------+


## Load Tables with Load Command <br>
In general this will NOT work because of authorisation issues

In [ ]:
# Download two CSV files from G-Drive into the Colab VM
!gdown https://drive.google.com/uc?id=1WxuruueFwMTjmPb-DvvVnGYB78Ty4OkP
!gdown https://drive.google.com/uc?id=1j49sqYHv0UoskVJDzNObWxECFF7ySsld

In [ ]:
#NOTE SECURITY WARNING
#Passing passwords in the command line is inherently risky and SHOULD NOT BE USED for any non-trivial work
#---------------------------------------------------------------------
!mysql  -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e "   \
LOAD DATA LOCAL INFILE 'SQLCourse2 - ItemsOrdered.csv' \
ignore \
INTO  TABLE itemsordered  \
FIELDS TERMINATED BY ',' enclosed by '\"' LINES TERMINATED BY '\r\n' IGNORE 1 LINES  \
(customerid,@orderdate, item, quantity, price) \
set orderdate = STR_TO_DATE(@orderdate,'%d-%b-%Y');\
"

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 3948 (42000) at line 1: Loading local data is disabled; this must be enabled on both the client and server sides


## Load Table with PhP MyAdmin 

CSV file can be uploaded  using the import command of the PhP MyAdmin function <br>
However for this, the files have to 'cleaned up'

In [ ]:
!mysql -h '$hostName' -P 3306 -u '$userName' -p'$passWord' '$dbName' -e 'select * from customers;'

mysql: [Warning] Using a password on the command line interface can be insecure.
+------------+-----------+----------+--------------+----------------+
| customerid | firstname | lastname | city         | state          |
+------------+-----------+----------+--------------+----------------+
| 10101      | John      | Gray     | Lynden       | Washington     |
| 10298      | Leroy     | Brown    | Pinetop      | Arizona        |
| 10299      | Elroy     | Keller   | Snoqualmie   | Washington     |
| 10315      | Lisa      | Jones    | Oshkosh      | Wisconsin      |
| 10325      | Ginger    | Schultz  | Pocatello    | Idaho          |
| 10329      | Kelly     | Mendoza  | Kailua       | Hawaii         |
| 10330      | Shawn     | Dalton   | Cannon Beach | Oregon         |
| 10338      | Michael   | Howell   | Tillamook    | Oregon         |
| 10339      | Anthony   | Sanchez  | Winslow      | Arizona        |
| 10408      | Elroy     | Cleaver  | Globe        | Arizona        |
| 10410  

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
dfItems = pd.read_csv('SQLCourse2 - ItemsOrdered.csv')
dfItems.head()

,customerid,order_date,item,quantity,price
0,10330,30-Jun-1999,Pogo stick,1,28.0
1,10101,30-Jun-1999,Raft,1,58.0
2,10298,01-Jul-1999,Skateboard,1,33.0
3,10101,01-Jul-1999,Life Vest,4,125.0
4,10299,06-Jul-1999,Parachute,1,1250.0


In [ ]:
# The order_date is changed to the proper MySQL format
dfItems['order_date'] = dfItems['order_date'].apply(lambda s: datetime.strptime(s,'%d-%b-%Y').strftime("%Y-%m-%d"))
dfItems.head()



,customerid,order_date,item,quantity,price
0,10330,1999-06-30,Pogo stick,1,28.0
1,10101,1999-06-30,Raft,1,58.0
2,10298,1999-07-01,Skateboard,1,33.0
3,10101,1999-07-01,Life Vest,4,125.0
4,10299,1999-07-06,Parachute,1,1250.0


In [ ]:
#convert modified dataframe to csv file for PhP MyAdmin Use
dfItems.to_csv('rItemsOrders.csv', index=False, header=False)

In [ ]:
# In this case, no date conversion is required, only header removed
dfCustomers = pd.read_csv('SQLCourse2 - Customers.csv')
dfCustomers.to_csv('rCustomers.csv', index=False, header=False)

# Pythons Functions for SQL commands

In [ ]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb

In [ ]:
#To run any non-SELECT SQL command
def runCMD (loc, DDL):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
    
    myCursor = DBConn.cursor()
    retcode = myCursor.execute(DDL) 
    print (retcode)
    DBConn.commit()
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (loc, CMD):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
    
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

## Test SQL calls from Python

In [ ]:
# Set the location to whichever Database you need to access
#
# --------------------------------
#
#location = 'localDB'
location = 'remoteDB'

In [ ]:
#runSELECT(location,'select * from customers')
runSELECT(location,'select * from itemsordered limit 5')

,customerid,orderdate,item,quantity,price
0,10330,1999-06-30,Pogo stick,1,28.0
1,10101,1999-06-30,Raft,1,58.0
2,10298,1999-07-01,Skateboard,1,33.0
3,10101,1999-07-01,Life Vest,4,125.0
4,10299,1999-07-06,Parachute,1,1250.0


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)